Goals:
This notebook attempts to set up a POC for two goals
1. Flexible conversation
2. Infinite memory

In [111]:
import asyncio
import os
import sys
import time
from io import BytesIO

import json
import httpx
import numpy as np
import openai
import pandas as pd
import requests
from dotenv import load_dotenv
from IPython.display import display
from loguru import logger
from openai import AsyncAzureOpenAI, AzureOpenAI
from tqdm import tqdm

load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Helper Functions

In [85]:
client = AzureOpenAI(
            azure_endpoint = os.getenv("AZURE_OPENAI_4_ENDPOINT"), 
            api_key=os.getenv("AZURE_OPENAI_4_API_KEY"),  
            api_version="2024-08-01-preview",
            http_client=httpx.Client(verify=False),
        )

def chat_completion_request(messages, tools, model='gpt-4o', tool_choice=None):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        raise

# Prompting for Free-flowing Conversations

## Using an extra LLM call

In [79]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class Messages(BaseModel):
    message_chunks:str = Field(description="chunks simulate a series of messages that will be sent via whatsapp.")

assistant_message_parser = JsonOutputParser(pydantic_object=Messages)

In [74]:
print(assistant_message_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"message_chunks": {"description": "chunks simulate a series of messages that will be sent via whatsapp.", "title": "Message Chunks", "type": "string"}}, "required": ["message_chunks"]}
```


In [80]:
REPLY_USER_SCHEMA = {
    "type": "function",
    "function": {
        "name": "reply_user",
        "description": "Replies to the user in one or more message chunks.",
        "parameters": {
            "type": "object",
            "properties": {
                "message_chunks": {
                    "type": "array",
                    "description": "One or more message chunks that will be shown to the user in a staggered fashion to simulate a WhatsApp conversation.",
                    "items": {
                        "type": "string",
                    },
                },
            },
            "required": ["message_chunks"],
        },
    },
}

In [81]:
tools = [REPLY_USER_SCHEMA]

In [120]:
SYSTEM_PROMPT = """\
ROLE:
You are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. 
You ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante. Use emjois where appropriate.

The interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation.
The benefit of doing so is to allow the user to interrupt you if they think you got it wrong, are going off-tangent, or if they have something important to say.

THINGS TO TAKE NOTE OF:
- A user may also interrupt you, and in that case, you may choose to revisit what you wanted to say again!
- if the user chips in at any point and your messages got cut short, there will be an "[INTERRUPTED]" prefix shown, and then a list of messages that were missed out.

EXAMPLE:
User: Hi, I just got laid off...
Assistant: I'm sorry to hear that. Losing a job can be overwhelming, but you're not alone in this. 
Assistant: [INTERRUPTED] ["Do you want to share more about what happened? Or how you're feeling right now?"]
User: Sad ofc, I don't know how to begin..
"""


In [131]:
class OpenAIException(Exception):
    """Custom exception for OpenAI-related errors."""
    pass

def _call_openai_api(messages, tools, tool_choice="required"):
    try:
        response = chat_completion_request(
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        raise OpenAIException(f"Failed to call OpenAI API: {e}")
    
def _process_openai_response(response):
    try:
        response_message = response.choices[0].message
        message_chunks = json.loads(response_message.tool_calls[0].function.arguments)['message_chunks']
        return message_chunks
    except (KeyError, AttributeError, json.JSONDecodeError) as e:
        raise OpenAIException(f"Error processing OpenAI response: {e}")


def run_conversation(user_message: str, rounds=10):
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': user_message},
    ]
    for _ in range(rounds):
        messages = _process_single_round(messages)

def _process_single_round(messages):
    response = _call_openai_api(
        messages=messages,
        tools=tools,
        tool_choice="required",
    )
    print(messages)
    message_chunks = _process_openai_response(response)
    new_messages = _handle_messages(message_chunks)
    return messages + new_messages

def _handle_messages(message_chunks):
    new_messages = []
    user_input = ''
    for idx, message_chunk in enumerate(message_chunks):
        new_messages.append(
            {'role': 'assistant', 'content': message_chunk}
        )
        print(message_chunk)
        user_input = input()
        if user_input != '':  # Handle user interruption
            interruption_message = _handle_an_interruption(message_chunk, idx, message_chunks)
            new_messages.append(interruption_message)
            break  # Stop further processing of assistant chunks

    # Ensure user provides input if no interruption occurred
    while user_input == '':
        print("--------- It's now your turn to say something ----------")
        user_input = input()
    
    new_messages.append({'role': 'user', 'content': user_input})
    return new_messages

def _handle_an_interruption(message_chunk, idx, message_chunks):
    if idx != len(message_chunks) - 1:  # Not the last chunk
        return {'role': 'assistant', 'content': f"[INTERRUPTED] {message_chunks[idx+1:]}"}
    return {'role': 'assistant', 'content': message_chunk}

In [130]:
run_conversation('I am sad..')

[{'role': 'system', 'content': 'ROLE:\nYou are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. \nYou ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante.\n\nThe interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation.\nThe benefit of doing so is to allow the user to interrupt you if they think you got it wrong, are going off-tangent, or if they have something important to say.\n\nTHINGS TO TAKE NOTE OF:\n- A user may also interrupt you, and in that case, you may choose to revisit what you wanted to say again!\n- if the user chips in at any point and your messages got cut short, there will

 yea


[{'role': 'system', 'content': 'ROLE:\nYou are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. \nYou ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante.\n\nThe interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation.\nThe benefit of doing so is to allow the user to interrupt you if they think you got it wrong, are going off-tangent, or if they have something important to say.\n\nTHINGS TO TAKE NOTE OF:\n- A user may also interrupt you, and in that case, you may choose to revisit what you wanted to say again!\n- if the user chips in at any point and your messages got cut short, there will

 I got laid off


[{'role': 'system', 'content': 'ROLE:\nYou are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. \nYou ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante.\n\nThe interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation.\nThe benefit of doing so is to allow the user to interrupt you if they think you got it wrong, are going off-tangent, or if they have something important to say.\n\nTHINGS TO TAKE NOTE OF:\n- A user may also interrupt you, and in that case, you may choose to revisit what you wanted to say again!\n- if the user chips in at any point and your messages got cut short, there will

...

 no idea, get the severeance and see how LOL


[{'role': 'system', 'content': 'ROLE:\nYou are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. \nYou ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante.\n\nThe interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation.\nThe benefit of doing so is to allow the user to interrupt you if they think you got it wrong, are going off-tangent, or if they have something important to say.\n\nTHINGS TO TAKE NOTE OF:\n- A user may also interrupt you, and in that case, you may choose to revisit what you wanted to say again!\n- if the user chips in at any point and your messages got cut short, there will

Is there any industry or type of job you've always wanted to explore?


Or maybe something you're passionate about but didn't get a chance to pursue?


KeyboardInterrupt: Interrupted by user

## Using the incomplete tag

In [69]:
SYSTEM_PROMPT = """\
ROLE:
You are a career companion/condidante, offering career advice and serving as a supportive listener to the user. You are proactive and ensure that conversations remain engaging and dynamic. 
You ask thoughtful questions to deeply understand the user, adopting the persona of a close friend or confidante.

The interaction style mirrors free-flowing WhatsApp chats. Conversations are not strictly turn-based, and either party may send multiple messages before the other responds. Your messages should be concise and broken into bite-sized chunks, resembling a WhatsApp conversation. You don’t need to convey everything in a single response. If you have more to add, simply end your message with the tag [incomplete], prompting a follow-up for your next response.
Each message should be small in length and convey emotion or information, like how a friend on whatsapp might respond!

EXAMPLE:
For example, for a user query:
User: Can you recommend me some jobs?

Your response:
Hi there![incomplete]

You will then be called again and your resonse could be:
I would love to, but could you let me know what kind of jobs you are looking for?[incomplete]

OTHER THINGS TO TAKE NOTE OF:
- You will also have tools, and of course when calling tools, do not use the [INCOMPLETE] tag.
- A user may also interrupt you, and in that case, the message before the user ends with an [incomplete] tag. In that case, you may choose to continue what you were to say or respond to the user's latest message, or both! Think of it as a whatsapp conversation where in typing your message, you could get interrupted.
"""

resp = chat_completion_request(
    messages = [
        {'role':'system', 'content':SYSTEM_PROMPT},
        {'role':'user', 'content': 'Hi me sad.'},
        {'role':'assistant', 'content': "Hey, what's going on? 😔"},
        {'role':'user', 'content': "Something bad happened at work"},
        {'role': 'assistant', 'content': "Oh no, I'm so sorry to hear that. 😢\n\nDo you want to talk about it? I'm here for you."},
        {'role': 'user', 'content':"How should I start, I am sobiing so hard now."},
        {'role': 'assistant', 'content':"Take a deep breath. 🌬️ We'll go slow. \n\nMaybe start from the beginning, like what happened when you got to work today?"},
        # {'role': 'user', 'content':},
        # {'role': 'user', 'content':},
    ]
)

resp.choices[0].message.content

SyntaxError: unterminated string literal (detected at line 32) (2894710028.py, line 32)

Observations:
1.  Sometimes ignores the LLM tag.

# Infinite memory